In [1]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import torch.nn.functional as F
from tqdm import tqdm
from IPython.display import Audio
from scipy.io import wavfile
import soundfile
import librosa
import torch
import random
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Model,Wav2Vec2Config, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

In [2]:
import logging
def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger

In [3]:
# Compute a binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels.
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Compute binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier labels.
def compute_one_vs_rest_confusion_matrix(labels, outputs):
    assert(np.shape(labels) == np.shape(outputs))
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients, num_classes = np.shape(labels)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_patients):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    num_patients, num_classes = np.shape(labels)

    A = compute_one_vs_rest_confusion_matrix(labels, outputs)

    f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            f_measure[k] = float('nan')

    if np.any(np.isfinite(f_measure)):
        macro_f_measure = np.nanmean(f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, f_measure

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    num_patients, num_classes = np.shape(labels)
    A = compute_confusion_matrix(labels, outputs)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    accuracy_classes = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(A[:, i]) > 0:
            accuracy_classes[i] = A[i, i] / np.sum(A[:, i])
        else:
            accuracy_classes[i] = float('nan')

    return accuracy, accuracy_classes

# Compute accuracy.
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

In [4]:
# from torchmetrics import F1
# from torchmetrics import Accuracy

from sklearn.metrics import f1_score

In [5]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument('--train_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/',     help='train feature path')
parser.add_argument('--valid_feature_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation_wav2vec2/',     help='validation feature path')
parser.add_argument('--output_folder',    default='./output_folder_LCNN',     help='output folder')
parser.add_argument("--mode",default="murmur", type=str, help="mode : murmur/outcome")
parser.add_argument("--lr",default=1e-3, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=32 ,type=int)
parser.add_argument("--minibatchsize_valid", default=8, type=int)
parser.add_argument("--train_num_workers", default=8, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_LCNN', type=str)
parser.add_argument("--logdir", default='./log_hmd_LCNN/', type=str)
parser.add_argument("--model_name", default='hmd_LCNN_model_murmur', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=80, type=int)
parser.add_argument("--gpu", default="0", type=str)




warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [6]:
def read_recording(wav_fn, audio_length= 20, sr= 4000):

    recording, frequency = librosa.load(wav_fn, sr = sr)
    
    recording = recording*1.0


    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording, frequency

In [7]:
from tqdm import tqdm
import pickle
import torch.nn as nn
wav_files = glob.glob(args.train_data_folder + '*.wav')
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
CTC = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
CTC.lm_head = nn.Identity()
# for i in tqdm(range(len(wav_files))):
#     current_recording = read_recording(wav_files[i])
# #     current_recording = current_recording.reshape(1,80000)
#     input_values = processor(current_recording, return_tensors="pt", padding="longest").input_values
#     feature = CTC(input_values).logits
#     feature.requires_grad = False
#     fnm = wav_files[i].split('/')[-1].replace('wav','pickle')
#     with open('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/'+ fnm ,'wb') as fw:
#         pickle.dump(feature,fw)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
CTC

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [13]:
wav_files = glob.glob(args.train_data_folder + '*.wav')
for i in tqdm(range(len(wav_files))):
    current_recording, frequency = read_recording(wav_files[i])
    current_recording = current_recording.reshape(1,80000)
#     input_values = processor(current_recording, return_tensors="pt", padding="longest").input_values
    feature = CTC(torch.tensor(current_recording)).logits
#     feature.requires_grad = False
    fnm = wav_files[i].split('/')[-1].replace('wav','pickle')
    with open(args.train_feature_folder + fnm ,'wb') as fw:
        pickle.dump(feature,fw)

100%|███████████████████████████████████████| 2532/2532 [07:31<00:00,  5.61it/s]


In [15]:
wav_files = glob.glob(args.valid_data_folder + '*.wav')
for i in tqdm(range(len(wav_files))):
    current_recording, frequency = read_recording(wav_files[i])
    current_recording = current_recording.reshape(1,80000)
    feature = CTC(torch.tensor(current_recording)).logits
#     feature.requires_grad = False
    fnm = wav_files[i].split('/')[-1].replace('wav','pickle')
    with open(args.valid_feature_folder+ fnm ,'wb') as fw:
        pickle.dump(feature,fw)

100%|█████████████████████████████████████████| 631/631 [01:41<00:00,  6.21it/s]


In [6]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class LCNN(nn.Module):

    def __init__(self,mode):

        super(LCNN, self).__init__()
        '''input : (B,1,F,T)'''
        
        self.mode = mode
        
        self.conv2d_1  = nn.Conv2d(1, 32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        self.conv2d_2  = nn.Conv2d(1, 32, kernel_size=(5,5), stride=(1,1), padding=(2,2))
        
        self.max2d_1 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.conv2d_3  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_4  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_1 = nn.BatchNorm2d(32)
        
        self.conv2d_5  = nn.Conv2d(32, 48, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_6  = nn.Conv2d(32, 48, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.max2d_2 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        self.bn2d_2 = nn.BatchNorm2d(48)
        
        
        self.conv2d_7  = nn.Conv2d(48, 48, kernel_size=(1,1), stride=(1,1))
        self.conv2d_8  = nn.Conv2d(48, 48, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_3 = nn.BatchNorm2d(48)
        
        self.conv2d_9  = nn.Conv2d(48, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_10  = nn.Conv2d(48, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.max2d_3 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.conv2d_11  = nn.Conv2d(64, 64, kernel_size=(1,1), stride=(1,1))
        self.conv2d_12  = nn.Conv2d(64, 64, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_4 = nn.BatchNorm2d(64)
        
        self.conv2d_13  = nn.Conv2d(64, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2d_14  = nn.Conv2d(64, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.bn2d_5 = nn.BatchNorm2d(32)
        
        
        self.conv2d_15  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_16  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.bn2d_6 = nn.BatchNorm2d(32)
        
        
        self.conv2d_17  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        self.conv2d_18  = nn.Conv2d(32, 32, kernel_size=(1,1), stride=(1,1))
        
        self.max2d_4 = nn.MaxPool2d(kernel_size =(2,2), stride=(2,2),padding = 0)
        
        self.adaptavg = nn.AdaptiveAvgPool2d(1)
        
        
#        self.fc1 = nn.Linear(64, 128)
        
        if self.mode == 'murmur' :
            self.fc_mur = nn.Sequential(
                                        nn.Linear(32,1),
                                        nn.Sigmoid()
                                        )
        elif self.mode == 'outcome' :
            self.fc_out = nn.Sequential(
                                        nn.Linear(128,32),
                                        nn.ReLU(),
                                        nn.Linear(32,1),
                                        nn.Sigmoid()
                                        )
        
        
    def forward(self, input):
        
        x1 = self.conv2d_1(input)
        x2 = self.conv2d_2(input)
        max1 = torch.maximum(x1,x2)
        x = self.max2d_1(max1)
        
        x3 = self.conv2d_3(x)
        x4 = self.conv2d_4(x)
        max2 = torch.maximum(x3,x4)
        x = self.bn2d_1(max2)
        
        x5 = self.conv2d_5(x)
        x6 = self.conv2d_6(x)
        max3 = torch.maximum(x5,x6)
        x = self.max2d_2(max3)
        x = self.bn2d_2(x)
        
        x7 = self.conv2d_7(x)
        x8 = self.conv2d_8(x)
        max4 = torch.maximum(x7,x8)
        x = self.bn2d_3(max4)
        
        x9 = self.conv2d_9(x)
        x10 = self.conv2d_10(x)
        max5 = torch.maximum(x9,x10)
        x = self.max2d_3(max5)
        
        x11 = self.conv2d_11(x)
        x12 = self.conv2d_12(x)
        max6 = torch.maximum(x11,x12)
        x = self.bn2d_4(max6)
        
        x13 = self.conv2d_13(x)
        x14 = self.conv2d_14(x)
        max7 = torch.maximum(x13,x14)
        x = self.bn2d_5(max7)
        
        x15 = self.conv2d_15(x)
        x16 = self.conv2d_16(x)
        max8 = torch.maximum(x15,x16)
        x = self.bn2d_6(max8)
        
        x17 = self.conv2d_17(x)
        x18 = self.conv2d_18(x)
        max9 = torch.maximum(x17,x18)
        x = self.max2d_4(max9)
        
        x = self.adaptavg(x)
        x = x.view(x.size(0),-1)
        
#        x = self.fc1(x)
        
        
        if self.mode == 'murmur' :
            out = self.fc_mur(x)
        elif self.mode == 'outcome' :
            out = self.fc_out(x)
#        out = {'murmur':out1, 'outcome':out2}
        
        return out

In [8]:
feature_files = glob.glob(args.train_feature_folder + '*.pickle')
murmur_classes = ['Present', 'Absent']
murmur_weight = []
murmur_label = []
for i in range(len(feature_files)):
    filename = feature_files[i]
    current_patient_id = filename.split('/')[-1].split('_')[0]
    current_txt_file = args.train_data_folder + str(current_patient_id) + '.txt'
    with open(current_txt_file, 'r') as f:
        current_patient_data = f.read()
    if get_murmur(current_patient_data)==murmur_classes[0]:
        murmur_weight.append(4)
        murmur_label.append(1)
    elif get_murmur(current_patient_data)==murmur_classes[1]:
        murmur_weight.append(1)
        murmur_label.append(0)
    else:
        murmur_weight.append(1)
        murmur_label.append(0)

In [7]:
from torch.utils.data import Dataset,DataLoader, RandomSampler
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
import librosa
import glob
import math
import pickle





# def read_recording(wav_fn, audio_length= 20, sr= 4000):  #20초 

#     recording, frequency = load_wav_file(wav_fn)


#     length = audio_length*sr
#     if recording.shape[0] <= length:
#         shortage = length - recording.shape[0]
#         recording = np.pad(recording, (0, shortage), 'wrap')
#     start_frame = np.int64(random.random()*(recording.shape[0]-length))
#     recording = recording[start_frame:start_frame + length] 

#     return recording



class train_loader(Dataset):
    def __init__(self,data_folder, feature_folder, **kwargs):
        self.data_folder = data_folder
        self.feature_folder = feature_folder
        
        
        self.feature_file = glob.glob(feature_folder + '*.pickle')
        self.wav_files = glob.glob(data_folder + '*.wav')
        self.num_feature_files = len(self.feature_file)
        
        self.murmur_classes = ['Present', 'Absent']
        self.num_murmur_classes = len(self.murmur_classes)
        self.outcome_classes = ['Abnormal', 'Normal']
        self.num_outcome_classes = len(self.outcome_classes)
        
        self.age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
        self.recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
        
        
        

        
        murmur_lable = np.zeros(self.num_feature_files, dtype=np.float32)
        outcome_lable = np.zeros(self.num_feature_files, dtype=np.float32)
        examples = {}
        for i in range(self.num_feature_files):
            filename = self.feature_file[i]
            current_patient_id = filename.split('/')[-1].split('_')[0]
            current_txt_file = self.data_folder + str(current_patient_id) + '.txt'
            with open(current_txt_file, 'r') as f:
                current_patient_data = f.read()
            current_loc = filename.split('/')[-1].split('.')[0].split('_')[1]
            if filename.split('/')[-1].split('.')[0]  not in examples.keys():
                current_feature_data = filename.split('/')[-1].split('.')[0]
                with open(filename, 'rb') as fr:
                    feature = pickle.load(fr)
                examples[current_feature_data] = {
                    "id" : current_patient_id,
                    "txt_fn" : current_txt_file,   #txt,
                    "feature" : feature,  #wav2vev2
                    "num_location": get_num_locations(current_patient_data),
                    "frequency" : get_frequency(current_patient_data),
                    "location" : current_loc,
                    "age" : get_age(current_patient_data),
                    "sex" : get_sex(current_patient_data),
                    "height" : get_height(current_patient_data),
                    "weight" : get_weight(current_patient_data),
                    "pregnancy_status" : get_pregnancy_status(current_patient_data),
                    "murmur" : get_murmur(current_patient_data),
                    "outcome" : get_outcome(current_patient_data)

                }
            if get_murmur(current_patient_data) == self.murmur_classes[0]:
                murmur_lable[i] = 1
            elif get_murmur(current_patient_data) == self.murmur_classes[1]:
                murmur_lable[i] = 0
            else :
                murmur_lable[i] = 0
                
            if get_outcome(current_patient_data) == self.outcome_classes[0]:
                outcome_lable[i] = 1
            elif get_outcome(current_patient_data) == self.outcome_classes[1]:
                outcome_lable[i] = 0
                
            
        self.examples = examples
        self.examples_list = list(examples.keys())
        self.murmur_lable = murmur_lable
        self.outcome_lable = outcome_lable
        
        
        
        
        
        if len(self.examples_list) != self.num_feature_files:
            raise ValueError('length is different!')
    def __len__(self):
        return len(self.examples_list)
    
        
    def __getitem__(self, index):
        
        # patient info
        current_feature_dict = self.examples[self.examples_list[index]]
        current_patient_data = load_patient_data(current_feature_dict["txt_fn"])

        
        # wav2vec2 feature
        
        feature = current_feature_dict["feature"]
        feature.requires_grad = False
        current_murmur = self.murmur_lable[index]
        current_outcome = self.outcome_lable[index]
        
        
#         #log mel feature
#         log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = current_recording.astype(np.float32),
#                                                          sr= 4000,
#                                                          n_mels=128,
#                                                          n_fft=400, 
#                                                          hop_length=128, 
#                                                          win_length=400))
        
#         # age
#         current_patient_age = current_wav_dict["age"]
#         current_age_group = np.zeros(6, dtype=np.float32)
#         if current_patient_age in self.age_classes:
#             j = self.age_classes.index(current_patient_age)
#             current_age_group[j] = 1.0
#         else :
#             current_age_group[5] = 1.0
            
            
#         # sex
#         sex = current_wav_dict["sex"]
#         sex_feature = np.zeros(2, dtype=np.float32)
#         if compare_strings(sex, 'Female'):
#             sex_feature[0] = 1.0
#         elif compare_strings(sex, 'Male'):
#             sex_feature[1] = 1.0
            
#         # height and weight.
#         height = current_wav_dict["height"]
#         weight = current_wav_dict["weight"]
        
#         ## simple impute
#         if math.isnan(height) :
#             height = 110.846  #mean
#         if math.isnan(weight) :
#             weight = 23.767   #mean
            
#         height_weight = np.array([height, weight], dtype=np.float32)
        
        
#         # Extract pregnancy
#         preg_feature = np.zeros(2, dtype=np.float32)
#         is_pregnant = current_wav_dict["pregnancy_status"]
#         if is_pregnant == True:
#             preg_feature[0] = 1.0
#         elif is_pregnant == False:
#             preg_feature[1] = 1.0

#         # Extract location
#         location = current_wav_dict["location"]
#         num_recording_locations = len(self.recording_locations)
#         loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
#         if location in self.recording_locations:
#             j = self.recording_locations.index(location)
#             loc_feature[j] = 1.0


        

        # label
        
#         current_murmur = np.zeros(self.num_murmur_classes, dtype=np.float32)
#         murmur = get_murmur(current_patient_data)
#         if murmur in self.murmur_classes:
#             j = self.murmur_classes.index(murmur)
#             current_murmur[j] = 1
#         else:
#             current_murmur[1] = 1   #unknown absent에 합침


#         current_outcome = np.zeros(self.num_outcome_classes, dtype=np.float32)
#         outcome = get_outcome(current_patient_data)
#         if outcome in self.outcome_classes:
#             j = self.outcome_classes.index(outcome)
#             current_outcome[j] = 1
            


        
        return feature, current_murmur, current_outcome

In [14]:
feature_file = glob.glob(args.train_feature_folder + '*.pickle')

In [26]:
feature_file[34]

'/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train_wav2vec2/68632_TV.pickle'

In [27]:
load_patient_data('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/68632.txt')

'68632 3 4000\nAV 68632_AV.hea 68632_AV.wav 68632_AV.tsv\nPV 68632_PV.hea 68632_PV.wav 68632_PV.tsv\nTV 68632_TV.hea 68632_TV.wav 68632_TV.tsv\n#Age: Child\n#Sex: Female\n#Height: 114.0\n#Weight: 26.2\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: 50384\n'

In [25]:
trainset[34]

(tensor([[[-4.2646e-02,  1.0644e-02,  5.1547e-02,  ..., -1.8400e-01,
           -3.0286e-02, -9.6961e-02],
          [-3.5525e-02,  4.7679e-03,  5.2417e-02,  ..., -1.7215e-01,
           -2.9180e-02, -1.0286e-01],
          [-4.2010e-02,  7.4536e-03,  4.2672e-02,  ..., -1.8307e-01,
           -2.9273e-02, -9.8510e-02],
          ...,
          [ 2.9015e-03,  2.2511e-04, -3.1314e-02,  ..., -1.8359e-01,
           -1.4312e-02, -7.5786e-02],
          [ 7.9736e-02,  3.3385e-02, -1.4494e-01,  ..., -2.6928e-01,
           -1.4058e-02,  5.1259e-02],
          [-8.9012e-02,  3.5963e-02,  5.1406e-02,  ..., -2.1893e-01,
           -2.8922e-02, -5.7201e-02]]]),
 0.0,
 1.0)

In [8]:
# Find data files.
# if verbose >= 1:
#     print('Finding data files...')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
    
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

# Find the patient data files.
patient_files = find_patient_files(args.train_data_folder)
num_patient_files = len(patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')

# Create a folder for the model if it does not already exist.
os.makedirs(args.model_path, exist_ok=True)
os.makedirs(args.logdir, exist_ok=True)

open(args.logdir + 'score.txt', "a+")
score_file = open(args.logdir + 'score.txt', "a+")


model_folder = args.model_path
log_dir = args.logdir
logger = get_logger(log_dir + '/' + model_folder)

# Define the model
lcnn_model = LCNN("murmur")
lcnn_model = lcnn_model.cuda(device)

# Define the metrics
# accuracy = Accuracy(num_classes=2)
# f1 = F1(num_classes=2,average='micro')

# optimizer, loss
opt = torch.optim.Adam(lcnn_model.parameters(), lr=args.lr, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
criterion = torch.nn.BCELoss()


#data loader
trainset = train_loader(args.train_data_folder, args.train_feature_folder )
validset = train_loader(args.valid_data_folder, args.valid_feature_folder )
point_sampler = RandomSampler(trainset)

dataloader_train = DataLoader(dataset=trainset,
                    batch_size=args.minibatchsize_train,
                           sampler =point_sampler,
                           num_workers=args.train_num_workers)
dataloader_valid = DataLoader(dataset=validset,
                    batch_size=args.minibatchsize_valid,
                           num_workers=args.dev_num_workers)

Available devices  1


In [51]:
a = torch.ceil(torch.tensor([0.6,0.3,0.1])-0.5).numpy()

In [52]:
a[1] == 0.0

True

In [58]:
b = np.array([1,2,3])

b.concat(a)

AttributeError: 'numpy.ndarray' object has no attribute 'concat'

In [61]:
np.array([[1.0,0.0,0.0] if i ==1 else [0.0,0.0,1.0] for i in [1.0,0.0,1.0]])

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [69]:
next(iter(dataloader_train))[0].shape

torch.Size([32, 1, 249, 768])

In [12]:
x = torch.randn((32,1,200,700)).cuda()
lcnn_model(x).squeeze(1)

RuntimeError: CUDA out of memory. Tried to allocate 206.00 MiB (GPU 0; 10.75 GiB total capacity; 6.73 GiB already allocated; 99.38 MiB free; 6.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
murmur_classes = ['Present', 'Unknown','Absent']
outcome_classes = ['Abnormal', 'Normal']
if args.mode=='murmur':
    classes =  murmur_classes 
elif args.mode =='outcome':
    classes = outcome_classes


    
all_file_train = len(trainset)
dataloader_train_len = len(dataloader_train)




for iter_ in range(args.end_iter):
    train_pred_label,  train_gt_label = [], []
    start_time = time.time()
    running_loss = 0.0
    idx = 0
    lcnn_model.train()
    for feature, current_murmur, _ in tqdm(dataloader_train): ###current_outcome
        feature = feature.cuda()
#         current_age_group = current_age_group.cuda()
#         sex_feature = sex_feature.cuda()
#         height_weight = height_weight.cuda()
#         preg_feature = preg_feature.cuda()
#         loc_feature = loc_feature.cuda()
        current_murmur = current_murmur.cuda()
#         current_outcome = current_outcome.cuda()






        output_train = lcnn_model(feature)   # output
        output_train = output_train.squeeze(1)


        output_preds_labels = (torch.ceil(output_train-0.5)).data.cpu().numpy()
        train_pred_label.extend(list(output_preds_labels))
        train_gt_label.extend(list(current_murmur.data.cpu().numpy()))
        
#         murmur_weight = [4 if i==1 else 1 for i in list(current_murmur.data.cpu().numpy())]
        
#         criterion = torch.nn.BCELoss(weight = torch.tensor(murmur_weight).cuda())
        loss = criterion(output_train, current_murmur)
                        
        


        idx +=1              

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_loss += loss.item()

    train_f1 = f1_score(y_true= train_gt_label, y_pred= train_pred_label)
    
    
    scheduler.step()
    print("lr: ", opt.param_groups[0]['lr'])
    logger.info("Iteration:{0}, train loss = {1:.6f} ,train F1 = {2:.6f} ".format(iter_, 
                 running_loss/dataloader_train_len,train_f1))

    # eval
    all_file_valid = len(validset)
    dataloader_valid_len = len(dataloader_valid)
    lcnn_model.eval()

    with torch.no_grad():
        running_loss_dev = 0.0
        index = 0
        dev_pred_label,  dev_gt_label = [], []
        for feature,  current_murmur, _ in tqdm(dataloader_valid):  #current_outcome
            feature_dev = feature.cuda()
#             current_age_group_dev = current_age_group.cuda()
#             sex_feature_dev = sex_feature.cuda()
#             height_weight_dev = height_weight.cuda()
#             preg_feature_dev = preg_feature.cuda()
#             loc_feature_dev = loc_feature.cuda()
            current_murmur_dev = current_murmur.cuda()
#             current_outcome_dev = current_outcome.cuda()


            output_dev = lcnn_model(feature_dev)
            output_dev = output_dev.squeeze(1)

            
            
            output_preds_labels_dev = (torch.ceil(output_dev-0.5)).data.cpu().numpy()
            dev_pred_label.extend(list(output_preds_labels_dev))
            dev_gt_label.extend(list(current_murmur_dev.data.cpu().numpy()))
            
            
#             murmur_weight_dev = [4 if i==1 else 1 for i in list(current_murmur_dev.data.cpu().numpy())]
        
#             criterion = torch.nn.BCELoss(weight = torch.tensor(murmur_weight_dev).cuda())
            loss_dev = criterion(output_dev, current_murmur_dev)
            
            
            running_loss_dev += loss_dev.item()
                        

            index = index+1   
            
        dev_preds_onehot_label = np.array([[1.0,0.0,0.0] if i == 1 else [0.0,0.0,1.0] for i in dev_pred_label])    
        dev_gt_onehot_label = np.array([[1.0,0.0,0.0] if i == 1 else [0.0,0.0,1.0] for i in dev_gt_label])
            
        

        dev_f1 = f1_score(y_true= dev_gt_label, y_pred= dev_pred_label)
        weighted_acc = compute_weighted_accuracy(dev_gt_onehot_label, dev_preds_onehot_label, classes)

        



    logger.info("valid Epoch:%d, Train loss=%.4f, Valid loss=%.4f,Valid F1=%.4f, Valid weighted acc=%.4f" %( iter_,
            running_loss/dataloader_train_len, running_loss_dev/dataloader_valid_len, dev_f1,weighted_acc ))
    score_file.write("%d epoch, VALID LOSS %f, VALID F1 %2.2f%%, VALID WEIGHTED ACC%2.2f%%\n"%(iter_, 
                            running_loss_dev/dataloader_valid_len, dev_f1, weighted_acc ))
    score_file.flush()

    torch.save(lcnn_model.state_dict(), os.path.join(model_folder, "{}_{}.model".format(args.model_name, iter_)))

    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
    logger.info("*" * 50)


# if verbose >= 1:
#     print('Done.')

100%|███████████████████████████████████████████| 80/80 [00:17<00:00,  4.64it/s]

lr:  0.00091
08-12 01:34 [MainProcess, 18718] [INFO ]  Iteration:0, train loss = 0.434671 ,train F1 = 0.350000 



100%|███████████████████████████████████████████| 79/79 [00:01<00:00, 68.80it/s]

08-12 01:34 [MainProcess, 18718] [INFO ]  valid Epoch:0, Train loss=0.4347, Valid loss=0.5125,Valid F1=0.4024, Valid weighted acc=0.5727
08-12 01:34 [MainProcess, 18718] [INFO ]  Time used for each epoch training: 18.407928943634033 seconds.
08-12 01:34 [MainProcess, 18718] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 80/80 [00:16<00:00,  4.72it/s]

...

08-12 01:57 [MainProcess, 18718] [INFO ]  valid Epoch:78, Train loss=0.0176, Valid loss=0.6725,Valid F1=0.5208, Valid weighted acc=0.6354
08-12 01:57 [MainProcess, 18718] [INFO ]  Time used for each epoch training: 18.130603313446045 seconds.
08-12 01:57 [MainProcess, 18718] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 80/80 [00:16<00:00,  4.71it/s]

lr:  5.288242786788324e-07
08-12 01:58 [MainProcess, 18718] [INFO ]  Iteration:79, train loss = 0.021580 ,train F1 = 0.997925 



100%|███████████████████████████████████████████| 79/79 [00:01<00:00, 69.67it/s]

08-12 01:58 [MainProcess, 18718] [INFO ]  valid Epoch:79, Train loss=0.0216, Valid loss=0.6676,Valid F1=0.5236, Valid weighted acc=0.6363
08-12 01:58 [MainProcess, 18718] [INFO ]  Time used for each epoch training: 18.133819580078125 seconds.
08-12 01:58 [MainProcess, 18718] [INFO ]  **************************************************


In [11]:
output_preds_labels_dev

array([-0.,  1., -0., -0., -0., -0., -0.], dtype=float32)

In [18]:
current_murmur = np.zeros((1,2), dtype=int)

In [21]:
current_murmur

array([[0, 1]])

In [20]:
current_murmur[0][0] = 0
current_murmur[0][1] = 1

In [17]:
dev_pred_label.count(1)

58

In [22]:
torch.cuda.empty_cache()